In [4]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from FeatureExtractor import *

In [5]:
first, last = section_range_lookup['MD589']
dms = dict([(sc, DataManager(stack='MD589', section=sc, segm_params_id='tSLIC200', load_mask=False)) 
            for sc in range(first, last+1)])

In [6]:
labels = ['Pn', '12N', 'Gr', "7N", '7n', "5N", 'VLL', 'LVe', 'SuVe']

In [16]:
label_polygon = pickle.load(open(os.environ['REPO_DIR']+'/database/label_polygon.pkl', 'r'))

In [8]:
patch_dir = '/home/yuncong/CSHL_data_patches/'

In [10]:
h, w = dms[first].image_height, dms[sec].image_width

patch_size = 224
half_size = patch_size/2
stride = 56

grid_ys, grid_xs = np.meshgrid(np.arange(half_size, h-half_size, stride), 
                               np.arange(half_size, w-half_size, stride), 
                               indexing='ij')

In [17]:
for sec in range(first, last+1):
    labelmap = np.zeros((grid_ys.size,), np.uint8)

    for label_ind, label in enumerate(labels):
        if sec not in label_polygon[label]:
            continue
        vertices = label_polygon[label][sec]
        polygon_path = Path(vertices)

    #     xmin, ymin  = np.min(vertices, axis=0).astype(np.int)
    #     xmax, ymax  = np.max(vertices, axis=0).astype(np.int)
    #     width = xmax - xmin + 1
    #     height = ymax - ymin + 1

    #     xs, ys = np.meshgrid(np.arange(xmin, xmax+1), np.arange(ymin, ymax+1))
        indices_inside = np.where(polygon_path.contains_points(np.c_[grid_xs.flat, grid_ys.flat]))[0]
        labelmap[indices_inside] = label_ind + 1
        
    np.save(patch_dir+'/labelmap/labelmap_%d.npy' % sec, labelmap)

In [18]:
labelmap = {}
neighbor_labels = defaultdict(set)
landmark_positions = defaultdict(dict)
bg_positions = defaultdict(dict)

for sec in range(first, last+1):
    labelmap[sec] = np.load(patch_dir+'/labelmap/labelmap_%d.npy' % sec).reshape(grid_xs.shape)
    
    for label_ind, label in enumerate(labels):
        
        if label_ind + 1 in labelmap[sec]:
            ys, xs = np.where(labelmap[sec] == label_ind + 1)
            
            landmark_positions[sec][label] = np.c_[xs, ys]
            
            ymin = np.min(ys)
            ymax = np.max(ys)
            xmin = np.min(xs)
            xmax = np.max(xs)
            
            a = set(np.unique(labelmap[sec][max(0, ymin-500/stride):ymax+500/stride, 
                                            max(0, xmin-500/stride):xmax+500/stride])) - {0, label_ind+1}
            neighbor_labels[label_ind] |= set([i-1 for i in a])
            
            surround_ys, surround_xs = np.where(labelmap[sec][max(0, ymin-500/stride):ymax+500/stride, 
                                                              max(0, xmin-500/stride):xmax+500/stride] == 0)
            bg_positions[sec][label] = np.c_[surround_xs + max(0, xmin-500/stride),
                                             surround_ys + max(0, ymin-500/stride)]
            
#             print sec, label, set([labels[i-1] for i in a])
            
landmark_positions.default_factory = None
neighbor_labels.default_factory = None
bg_positions.default_factory = None

In [23]:
pickle.dump(landmark_positions, open(patch_dir+'landmark_positions.pkl', 'wb'))
pickle.dump(neighbor_labels, open(patch_dir+'neighbor_labels.pkl', 'wb'))
pickle.dump(bg_positions, open(patch_dir+'bg_positions.pkl', 'wb'))

In [14]:
# z = np.zeros_like(grid_xs)
# for label, vertices in bg_positions[106].iteritems():
#     for x, y in vertices:
#         z[y,x] = labels.index(label) + 1
# plt.matshow(z);

In [24]:
# nearby landmark classes
[(labels[label_ind], [labels[n] for n in neighbors]) for label_ind, neighbors in neighbor_labels.iteritems()]

[('Pn', []),
 ('12N', ['Gr']),
 ('Gr', ['12N']),
 ('7N', ['7n']),
 ('7n', ['7N', '5N', 'LVe']),
 ('5N', ['7n']),
 ('VLL', []),
 ('LVe', ['SuVe', '7n']),
 ('SuVe', ['LVe'])]

In [ ]:
for sec in range(first, last+1):
    
    sys.stderr.write('%d\n'%sec)
    
    if sec not in bg_positions and sec not in landmark_positions:
        dms[sec]._load_image(['rgb'])
    
    if sec in bg_positions:

        for label_ind, label in enumerate(labels):
            if label not in bg_positions[sec]: continue

            patches = []

            inds = bg_positions[sec][label]
            xs = grid_xs[inds[:,1], inds[:,0]]
            ys = grid_ys[inds[:,1], inds[:,0]]
            for i in range(xs.size):
                x = xs[i]
                y = ys[i]
                patch = dms[sec].image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
                patches.append(patch)

            patches_arr = np.array([img_as_ubyte(p) for p in patches], np.uint8)
            np.save(patch_dir+'/patches/patchesBg_%s_%d.npy'%(label, sec), patches_arr)
        
    if sec in landmark_positions:

        for label_ind, label in enumerate(labels):
            if label not in landmark_positions[sec]: continue

            patches = []

            inds = landmark_positions[sec][label]
            xs = grid_xs[inds[:,1], inds[:,0]]
            ys = grid_ys[inds[:,1], inds[:,0]]
            for i in range(xs.size):
                x = xs[i]
                y = ys[i]
                patch = dms[sec].image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
                patches.append(patch)

            patches_arr = np.array([img_as_ubyte(p) for p in patches], np.uint8)
            np.save(patch_dir+'/patches/patches_%s_%d.npy'%(label, sec), patches_arr)
                
    if sec not in bg_positions and sec not in landmark_positions:
        del dms[sec].image_rgb